In [11]:
from modules import goodreads, notion

# Provide the Goodreads URL to capture the metadata for
URL = "https://www.goodreads.com/book/show/55275019-machine-learning-design-patterns?ref=nav_sb_ss_1_111"

book = goodreads.get_book_metadata(URL, verbose=True)

Name: Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps
Image: https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1599583594i/55275019.jpg
Format: Paperback
Number of pages: 400
Language: English
ISBN: 9781098115784
Rating: 4.2
Number of ratings: 147
Number of reviews: 21
Authors: ['Valliappa Lakshmanan', 'Sara Robinson', 'Michael Munn']


In [12]:
notion.post_book_from_goodreads(book)

<Response [400]>

In [18]:
import requests
import json
NOTION_API_KEY="secret_F2r8vuOCteoio7aYUs2lS62ZuVEppxCwYghbnSXFrZA"
NOTION_DATABASE_ID="12896e3b2c4141ca83ed2993432f9750"

endpoint = "https://api.notion.com/v1/pages/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
    "Authorization": f"Bearer {NOTION_API_KEY}",
}
data = json.dumps(
    {
        "parent": {"database_id": NOTION_DATABASE_ID},
        "properties": {
            "Cover": {
                "type": "files",
                "files": [
                    {
                        "name": "Cover",
                        "type": "external",
                        "external": {"url": book.image},
                    }
                ],
            },
            "Status": {"type": "select", "select": None},
            "End": {"type": "date", "date": None},
            "Name": {
                "type": "title",
                "title": [{"type": "text", "text": {"content": book.name}}],
            },
            "Num Pages": {
                "type": "number",
                "number": book.num_pages,
            },
            "ISBN": {
                "type": "rich_text",
                "rich_text": [{"type": "text", "text": {"content": book.isbn}}],
            },
            "Author": {
                "type": "rich_text",
                "rich_text": [{"type": "text", "text": {"content": ", ".join(book.authors)}}],
            },
            "URL": {"type": "url", "url": book.url},
        },
    }
)

requests.request("POST", endpoint, headers=headers, data=data)

<Response [200]>